In [1]:
import pandas as pd
import numpy as np 
import wget
import zipfile
import os
from datetime import date
import datetime

In [110]:
# Dates with historical data and breaks
end_date    = date(2020, 10, 4)
#end_date    = date(2020, 4, 15)
start_date  = date(2020, 4, 12)
break_date  = date(2020, 10, 6)
dateseries  = [date.fromordinal(i) for i in range(start_date.toordinal(), end_date.toordinal()+1)]

# Empty dataset
columns     = ['id','DeathsR','CasesR']
mxr         = pd.DataFrame(index=dateseries,columns=columns)
mxr         = mxr.reset_index()
mxr         = mxr.rename(columns={"index": "date"})
mxr['date'] = pd.to_datetime(mxr['date'])
mxr['id']   = 0

mxr = mxr.reset_index()
del mxr['index']

In [111]:
# Download the entire raw data and build a new dataset with specific selections - as reported!!
# Data address: https://www.gob.mx/salud/documentos/datos-abiertos-152127
    
for i in dateseries:
    print('Working on: '+str(i), end="\r")

    # Extract year, month, day
    year = str(i.year-2000)
    if i.month<10:
        month = str(0)+str(i.month)
    else:
        month = str(i.month)
    if i.day<10:    
        day   = str(0)+str(i.day)
    else:   
        day   = str(i.day)

    # Download and extract file    
    filenamezip = 'datos_abiertos_covid19_'+day+'.'+month+'.20'+year+'.zip'
    if i.year<=2020:
        url = 'http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/'+month+'/'+filenamezip
    else:
        url = 'http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/20'+year+'/'+month+'/'+filenamezip
    filenamezip = wget.download(url)
    with zipfile.ZipFile(filenamezip, 'r') as zip_ref:
        zip_ref.extractall()

    # Read data and delete files
    filename = year+month+day+'COVID19MEXICO.csv'
    df = pd.read_csv(filename,encoding='latin1',low_memory=False)
    os.remove(filenamezip)
    os.remove(filename)

    # Data selection DEATHS
    if i<=break_date:
        s = df[
            (df['FECHA_DEF'] != '9999-99-99') & 
            (df['RESULTADO'] == 1) 
        ][['FECHA_DEF','RESULTADO']] .groupby("FECHA_DEF") .count() .cumsum()
        s.index= pd.to_datetime(s.index)
        idx = pd.date_range(start_date.strftime("%Y-%m-%d"), i.strftime("%Y-%m-%d"))
        s = s.reindex(idx).fillna(method='ffill').fillna(0)
        s = s.rename(columns={'RESULTADO': 'DeathsR'})
    else:
        s = df[
            (df['FECHA_DEF'] != '9999-99-99') & 
            (df['CLASIFICACION_FINAL'] <= 3)
        ][['FECHA_DEF','CLASIFICACION_FINAL']] .groupby("FECHA_DEF") .count() .cumsum()
        s.index= pd.to_datetime(s.index)
        idx = pd.date_range(start_date.strftime("%Y-%m-%d"), i.strftime("%Y-%m-%d"))
        s = s.reindex(idx).fillna(method='ffill').fillna(0)
        s = s.rename(columns={'CLASIFICACION_FINAL': 'DeathsR'})

    mxr.loc[(mxr['date']==pd.to_datetime(i)),'DeathsR'] = s['DeathsR'][-1]
    del s
    del idx
    
    # Data selection CASES
    if i<=break_date:
        s = df[
            (df['FECHA_SINTOMAS'] != '9999-99-99') & 
            (df['RESULTADO'] == 1) 
        ][['FECHA_SINTOMAS','RESULTADO']] .groupby("FECHA_SINTOMAS") .count() .cumsum()
        s.index= pd.to_datetime(s.index)+ pd.DateOffset(1)
        idx = pd.date_range(start_date.strftime("%Y-%m-%d"), i.strftime("%Y-%m-%d"))
        s = s.reindex(idx).fillna(method='ffill').fillna(0)
        s = s.rename(columns={'RESULTADO': 'CasesR'})
    else:
        s = df[
            (df['FECHA_SINTOMAS'] != '9999-99-99') & 
            (df['CLASIFICACION_FINAL'] <= 3)
        ][['FECHA_SINTOMAS','CLASIFICACION_FINAL']] .groupby("FECHA_SINTOMAS") .count() .cumsum()
        s.index= pd.to_datetime(s.index)+ pd.DateOffset(1)
        idx = pd.date_range(start_date.strftime("%Y-%m-%d"), i.strftime("%Y-%m-%d"))
        s = s.reindex(idx).fillna(method='ffill').fillna(0)
        s = s.rename(columns={'CLASIFICACION_FINAL': 'CasesR'})

    mxr.loc[(mxr['date']==pd.to_datetime(i)),'CasesR'] = s['CasesR'][-1]
    del s
    del idx 

    del df

    

In [112]:
mxr

,date,id,DeathsR,CasesR
0,2020-04-12,0,296,4661
1,2020-04-13,0,332,5014
2,2020-04-14,0,406,5399
3,2020-04-15,0,449,5847
4,2020-04-16,0,486,6297
5,2020-04-17,0,546,6875
6,2020-04-18,0,650,7497
7,2020-04-19,0,686,8261
8,2020-04-20,0,712,8772
9,2020-04-21,0,857,9501


In [113]:
# Save dataset 
mxr.to_csv('Data_Reported_all.csv', index=False)


In [107]:
# Get data as occurred

end_date    = date(2021, 9, 18)
#end_date    = date(2020, 5, 15)
start_date  = date(2020, 4, 12)
break_date  = date(2020, 10, 6)
dateseries  = [date.fromordinal(i) for i in range(start_date.toordinal(), end_date.toordinal()+1)]

# Empty dataset
columns     = ['id','DeathsO','CasesO']
mxo         = pd.DataFrame(columns=columns)

for i in [end_date]:
    print('Working on: '+str(i), end="\r")
    
    # Extract year, month, day
    year = str(i.year-2000)
    if i.month<10:
        month = str(0)+str(i.month)
    else:
        month = str(i.month)
    if i.day<10:    
        day   = str(0)+str(i.day)
    else:   
        day   = str(i.day)

    # Download and extract file    
    filenamezip = 'datos_abiertos_covid19_'+day+'.'+month+'.20'+year+'.zip'
    if i.year<=2020:
        url = 'http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/'+month+'/'+filenamezip
    else:
        url = 'http://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/20'+year+'/'+month+'/'+filenamezip
    filenamezip = wget.download(url)
    with zipfile.ZipFile(filenamezip, 'r') as zip_ref:
        zip_ref.extractall()

    # Read data and delete files
    filename = year+month+day+'COVID19MEXICO.csv'
    df = pd.read_csv(filename,encoding='latin1',low_memory=False)
    os.remove(filenamezip)
    os.remove(filename)

    # Municipality and entity

    # Data selection
    if i<=break_date:
        s = df[
            (df['FECHA_DEF'] != '9999-99-99') & 
            (df['RESULTADO'] == 1) 
        ][['FECHA_DEF','RESULTADO']] .groupby("FECHA_DEF") .count() .cumsum()
        s.index= pd.to_datetime(s.index)
        idx = pd.date_range(start_date.strftime("%Y-%m-%d"), i.strftime("%Y-%m-%d"))
        s = s.reindex(idx).fillna(method='ffill').fillna(0)
        s = s.rename(columns={'RESULTADO': 'DeathsO'})
    else:
        s = df[
            (df['FECHA_DEF'] != '9999-99-99') & 
            (df['CLASIFICACION_FINAL'] <= 3)
        ][['FECHA_DEF','CLASIFICACION_FINAL']] .groupby("FECHA_DEF") .count() .cumsum()
        s.index= pd.to_datetime(s.index)
        idx = pd.date_range(start_date.strftime("%Y-%m-%d"), i.strftime("%Y-%m-%d"))
        s = s.reindex(idx).fillna(method='ffill').fillna(0)
        s = s.rename(columns={'CLASIFICACION_FINAL': 'DeathsO'})
    del idx
    
    if i<=break_date:
        ss = df[
            (df['FECHA_SINTOMAS'] != '9999-99-99') & 
            (df['RESULTADO'] == 1) 
        ][['FECHA_SINTOMAS','RESULTADO']] .groupby("FECHA_SINTOMAS") .count() .cumsum()
        ss.index= pd.to_datetime(ss.index)+ pd.DateOffset(1)
        idx = pd.date_range(start_date.strftime("%Y-%m-%d"), i.strftime("%Y-%m-%d"))
        ss = ss.reindex(idx).fillna(method='ffill').fillna(0)
        ss = ss.rename(columns={'RESULTADO': 'CasesO'})
    else:
        ss = df[
            (df['FECHA_SINTOMAS'] != '9999-99-99') & 
            (df['CLASIFICACION_FINAL'] <= 3)
        ][['FECHA_SINTOMAS','CLASIFICACION_FINAL']] .groupby("FECHA_SINTOMAS") .count() .cumsum()
        ss.index= pd.to_datetime(ss.index)+ pd.DateOffset(1)
        idx = pd.date_range(start_date.strftime("%Y-%m-%d"), i.strftime("%Y-%m-%d"))
        ss = ss.reindex(idx).fillna(method='ffill').fillna(0)
        ss = ss.rename(columns={'CLASIFICACION_FINAL': 'CasesO'})

    s = s.join(ss)
    mxo = mxo.append(s)
    del s,ss,idx
    del df

mxo         = mxo.reset_index()
mxo         = mxo.rename(columns={"index": "date"}) 

In [109]:
mxo.head(30)

,date,CasesO,DeathsO,id
0,2020-04-12,11696,727.0,NaN
1,2020-04-13,12566,828.0,NaN
2,2020-04-14,13638,941.0,NaN
3,2020-04-15,14657,1058.0,NaN
4,2020-04-16,15850,1165.0,NaN
5,2020-04-17,17014,1308.0,NaN
6,2020-04-18,18355,1445.0,NaN
7,2020-04-19,19765,1627.0,NaN
8,2020-04-20,21024,1838.0,NaN
9,2020-04-21,23098,2055.0,NaN


In [128]:
# Put both together
mxr = pd.read_csv('Data_Reported_all.csv',encoding='latin1',low_memory=False)
mxr['date'] = mxr['date'].astype('datetime64[ns]')
mx = mxo.join(mxr[mxr['date']<='2020-10-05'][['CasesR','DeathsR']]).reset_index()
del mx['id']

In [131]:
mx.to_csv('Data_Reported_all_both.csv', index=False)